In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

import numpy as np

import PIL
from PIL import Image

import time

import os

# Extra imports
import io
from tensorflow.python.data.experimental import AUTOTUNE

In [ ]:
data_path = '/mnt/d/cardimagescans'
set_code = 'khm'

In [ ]:
def load_images_and_labels(datapath, setcode):
    directory = os.listdir(os.path.join(datapath, setcode))
    labels = [name[:-4] for name in directory]
    paths = [os.path.join(datapath, setcode, name) for name in directory]

    return paths, labels

In [ ]:
image_paths, image_labels = load_images_and_labels(data_path, set_code)

In [ ]:
def build_tfrecords(pathlist, labellist):
    with tf.io.TFRecordWriter('mtg.tfrecords') as writer:
        int_label = [i for i in range(len(labellist))]
        for path, label in zip(pathlist, int_label):
            image = Image.open(path)
            bytes_buffer = io.BytesIO()
            image.convert('RGB').save(bytes_buffer, 'JPEG')
            image_bytes = bytes_buffer.getvalue()

            bytes_feature = tf.train.Feature(bytes_list = tf.train.BytesList(value=[image_bytes]))
            class_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))

            example = tf.train.Example(
                features = tf.train.Features(feature={
                    'image': bytes_feature,
                    'class': class_feature
                })
            )
            writer.write(example.SerializeToString())
            image.close()

image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, [1040, 745, 3])
    return image, parsed_example["class"]

def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

In [ ]:
build_tfrecords(image_paths, image_labels)
print(len(image_paths))

In [ ]:

ds_train = load_and_extract_images('mtg.tfrecords')
print(ds_train)

In [ ]:
for image, label in ds_train.take(3):
    print(image)
    print(label)

In [ ]:
train_split = 0.6
validation_split = 0.2

num_train_images = int(len(image_paths) * train_split)
num_validation_images = int(len(image_paths) * validation_split)

train_dataset = ds_train.take(num_train_images)
validation_dataset = ds_train.skip(num_train_images).take(num_validation_images)
test_dataset = ds_train.skip(num_train_images + num_validation_images)

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Input(shape = (1040, 745, 3)))
model.add(layers.Conv2D(64, 7))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(128, 4))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(256, 4))
model.add(layers.MaxPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10))

optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 8
epochs = 10

model.fit(train_dataset.batch(batch_size),validation_data=validation_dataset.batch(batch_size), epochs=epochs)